# Deep Q-Network

In [1]:
#Imports
import gym
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import tensorflow as tf
from tensorflow import keras
import random

#Create Gym
from gym import wrappers
envCartPole = gym.make('CartPole-v1')
envCartPole.seed(50) #Set the seed to keep the environment consistent across runs

In [ ]:
EPISODES = 300
TRAIN_END = 0

def discount_rate(): #Gamma
    return 0.95

def learning_rate(): #Alpha
    return 0.001

def batch_size(): #Size of the batch used in the experience replay
    return 24

nS = envCartPole.observation_space.shape[0] #This is only 4
nA = envCartPole.action_space.n #Actions
batch_size = batch_size()

In [ ]:
class DeepQNetwork():
    def __init__(self, states, actions, alpha, gamma, epsilon,epsilon_min, epsilon_decay):
        self.nS = states
        self.nA = actions
        self.memory = deque([], maxlen=2500)
        self.alpha = alpha
        self.gamma = gamma
        #Explore/Exploit
        self.epsilon = epsilon
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.model = self.build_model()
        self.loss = []

    def build_model(self):
        model = keras.Sequential()
        model.add(keras.layers.Dense(24, input_dim=self.nS, activation='relu')) #[Input] -> Layer 1
        model.add(keras.layers.Dense(24, activation='relu')) #Layer 2 -> 3
        model.add(keras.layers.Dense(self.nA, activation='linear')) #Layer 3 -> [output]
        model.compile(loss='mean_squared_error', optimizer=keras.optimizers.Adam(lr=self.alpha))
        return model

    def action(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.nA) #Explore
        action_vals = self.model.predict(state) #Exploit:
        return np.argmax(action_vals[0])

    def test_action(self, state): #Exploit
        action_vals = self.model.predict(state)
        return np.argmax(action_vals[0])

    def store(self, state, action, reward, nstate, done):
        self.memory.append( (state, action, reward, nstate, done) )

    def experience_replay(self, batch_size):
        minibatch = random.sample( self.memory, batch_size )

        x = []
        y = []
        np_array = np.array(minibatch)
        st = np.zeros((0,self.nS)) #States
        nst = np.zeros( (0,self.nS) )#Next States
        for i in range(len(np_array)): #Creating the state and next state np arrays
            st = np.append( st, np_array[i,0], axis=0)
            nst = np.append( nst, np_array[i,3], axis=0)
        st_predict = self.model.predict(st) 
        nst_predict = self.model.predict(nst)
        index = 0
        for state, action, reward, nstate, done in minibatch:
            x.append(state)
            #Predict from state
            nst_action_predict_model = nst_predict[index]
            if done == True: 
                target = reward
            else:   #Non terminal
                target = reward + self.gamma * np.amax(nst_action_predict_model)
            target_f = st_predict[index]
            target_f[action] = target
            y.append(target_f)
            index += 1
        x_reshape = np.array(x).reshape(batch_size,self.nS)
        y_reshape = np.array(y)
        epoch_count = 1 #Epochs is the number or iterations
        hist = self.model.fit(x_reshape, y_reshape, epochs=epoch_count, verbose=0)

        for i in range(epoch_count):
            self.loss.append( hist.history['loss'][i] )
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [ ]:
dqn = DeepQNetwork(nS, nA, learning_rate(), discount_rate(), 1, 0.001, 0.995 )
#Training
rewards = [] #Store rewards for graphing
epsilons = [] # Store the Explore/Exploit
TEST_Episodes = 0
for e in range(EPISODES):
    state = envCartPole.reset()
    state = np.reshape(state, [1, nS]) # Resize to store in memory to pass to .predict
    tot_rewards = 0
    for time in range(210): #200 is when you "solve" the game. This can continue forever as far as I know
        action = dqn.action(state)
        nstate, reward, done, _ = envCartPole.step(action)
        nstate = np.reshape(nstate, [1, nS])
        tot_rewards += reward
        dqn.store(state, action, reward, nstate, done) # Resize to store in memory to pass to .predict
        state = nstate
        #done: CartPole fell.
        #time == 209: CartPole stayed upright
        if done or time == 209:
            rewards.append(tot_rewards)
            epsilons.append(dqn.epsilon)
            print("episode: {}/{}, score: {}, e: {}"
                  .format(e, EPISODES, tot_rewards, dqn.epsilon))
            break
        #Experience Replay
        if len(dqn.memory) > batch_size:
            dqn.experience_replay(batch_size)
    #If our current NN passes we are done
    #I am going to use the last 5 runs
    if len(rewards) > 5 and np.average(rewards[-5:]) > 195:
        #Set the rest of the EPISODES for testing
        TEST_Episodes = EPISODES - e
        TRAIN_END = e
        break

episode: 0/300, score: 17.0, e: 1
1/1 [==============================] - 0s 37ms/step
episode: 1/300, score: 17.0, e: 0.9558895783575597
1/1 [==============================] - 0s 25ms/step
episode: 2/300, score: 25.0, e: 0.8475428503023453
1/1 [==============================] - 0s 28ms/step
episode: 3/300, score: 15.0, e: 0.7901049725470279
1/1 [==============================] - 0s 27ms/step
episode: 4/300, score: 17.0, e: 0.7292124703704616
1/1 [==============================] - 0s 19ms/step
episode: 5/300, score: 21.0, e: 0.6596532430440636
1/1 [==============================] - 0s 30ms/step
episode: 6/300, score: 20.0, e: 0.5997278763867329
1/1 [==============================] - 0s 19ms/step
episode: 7/300, score: 10.0, e: 0.5732736268885887
1/1 [==============================] - 0s 19ms/step
episode: 8/300, score: 19.0, e: 0.5238143793828016
1/1 [==============================] - 0s 19ms/step
episode: 9/300, score: 10.0, e: 0.500708706245853
1/1 [==============================] - 0

In [ ]:
#Test the agent that was trained
#   In this section we ALWAYS use exploit don't train any more
for e_test in range(TEST_Episodes):
    state = envCartPole.reset()
    state = np.reshape(state, [1, nS])
    tot_rewards = 0
    for t_test in range(210):
        action = dqn.test_action(state)
        nstate, reward, done, _ = envCartPole.step(action)
        nstate = np.reshape( nstate, [1, nS])
        tot_rewards += reward
        #DON'T STORE ANYTHING DURING TESTING
        state = nstate
        #done: CartPole fell.
        #t_test == 209: CartPole stayed upright
        if done or t_test == 209:
            rewards.append(tot_rewards)
            epsilons.append(0) #We are doing full exploit
            print("episode: {}/{}, score: {}, e: {}"
                  .format(e_test, TEST_Episodes, tot_rewards, 0))
            break;

1/1 [==============================] - 0s 34ms/step


1/1 [==============================] - 0s 24ms/step
episode: 0/189, score: 210.0, e: 0
1/1 [==============================] - 0s 25ms/step
episode: 1/189, score: 210.0, e: 0
1/1 [==============================] - 0s 40ms/step
episode: 2/189, score: 210.0, e: 0
1/1 [==============================] - 0s 24ms/step
episode: 3/189, score: 210.0, e: 0
1/1 [==============================] - 0s 27ms/step
episode: 4/189, score: 210.0, e: 0
1/1 [==============================] - 0s 29ms/step
episode: 5/189, score: 210.0, e: 0
1/1 [==============================] - 0s 28ms/step
episode: 6/189, score: 210.0, e: 0
1/1 [==============================] - 0s 26ms/step
episode: 7/189, score: 210.0, e: 0
1/1 [==============================] - 0s 28ms/step
episode: 8/189, score: 210.0, e: 0
1/1 [==============================] - 0s 36ms/step
episode: 9/189, score: 210.0, e: 0
1/1 [==============================] - 0s 23ms/step
episode: 10/189, score: 210.0, e: 0
1/1 [==============================] - 0s 

In [1]:
rolling_average = np.convolve(rewards, np.ones(100)/100)

plt.figure(figsize=(8, 5))
plt.plot(rewards, label='Rewards')
plt.plot(rolling_average, color='black', label='Average Scores')
eps_graph = [200*x for x in epsilons]
plt.plot(eps_graph, color='g', linestyle='-', label='Epsilon (0.001 - 1.0)')
plt.title("Cartpole Experiment with DDQN")
plt.xlabel("Episodes")
plt.ylabel("Scores")
plt.xlim((0, EPISODES))
plt.ylim((0, 220))
plt.legend()
plt.show()
